# Optimised RAG Pipeline

## 1. Pipeline Architecture
### 2. Extract Queries from XLSX


1.   Read all sheets in the uploaded XLSX file.
2.   Combine them into a single DataFrame while preserving sheet names as headers.

3. Save the transformed DataFrame as a CSV file in a "processed_input" folder in the same bucket.


4. Process Queries : Read the transformed CSV file.
5. Process each query by:
  
  *   Checking Firestore for a cached response.

  *   Calling the Discovery Engine API if not found.

  *   Storing response embeddings in Firestore.

  *   Logging queries & responses into BigQuery.
  *  Add the responses as a new column.
    
  * Save the updated DataFrame in a "processed_output" folder in the same bucket.
  * Save Final Results

### 3. Upload the processed CSV file to the Cloud Storage bucket.


In [ ]:
!pip install google-cloud-aiplatform kfp google-cloud-storage google-cloud-firestore google-cloud-bigquery sentence-transformers requests pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 18.8 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.12.1-py3-none-any.whl size=366348 sha256=79b713dca26332d264fe966e569d87d8e589c5d0cc2483b8cd0b2fd12d418566
  Stored in directory: /root/.cache/pip/wheels/4c/a4/89/6f1fa2a3dae3976bc14d70e368e4064be8d4b0628af0ef7b85
  Created wheel for kfp-server-api: filename=kfp_server_api-2.4.0-py3-none-any.whl size=116526 sha256=a8ee38f1919017efe573c788b4dddfd39d83b5357246f91ea536d37b25573b58
  Stored in directory: /root/.cache/pip/wheels/8d/0f/30/65abccda2186c59a28fd37c13675dca68d4e3b9e1

# Install Dependencies

In [ ]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact, Dataset, Input, Output, component, Model)
from google.cloud import storage, firestore, bigquery
import pandas as pd
import requests
import json
import hashlib
from datetime import datetime
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize Sentence-BERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')


# Extract Queries from xlsx Component

In [ ]:
@dsl.component(base_image="python:3.9")
def extract_queries_from_xlsx(
    source_bucket: str, file_name: str, results_bucket: str
) -> str:
    import subprocess
    import sys
    subprocess.run([sys.executable, "-m", "pip", "install", "google-cloud-storage", "pandas", "openpyxl"], check=True)

    from google.cloud import storage
    import pandas as pd
    import os

    # Initialize Cloud Storage Client
    storage_client = storage.Client()
    source_bucket_ref = storage_client.bucket(source_bucket)
    results_bucket_ref = storage_client.bucket(results_bucket)
    blob = source_bucket_ref.blob(file_name)

    # Define Paths
    local_xlsx_path = f"/tmp/{file_name}"
    processed_file_name = file_name.replace('.xlsx', '.csv')
    processed_gcs_path = f"extracted_queries/{processed_file_name}"  # ✅ Save in `extracted_queries/`
    local_output_path = f"/tmp/{processed_file_name}"

    # Download XLSX File
    blob.download_to_filename(local_xlsx_path)
    print(f"Downloaded {file_name} to {local_xlsx_path}")

    # Read all sheets in the XLSX file
    xls = pd.ExcelFile(local_xlsx_path)
    all_data = []

    for sheet_name in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet_name)
        if not df.empty:
            df.insert(0, "Sheet_Name", sheet_name)  # ✅ Preserve Sheet Name
            all_data.append(df)

    if not all_data:
        raise ValueError("No data found in any sheets.")

    combined_df = pd.concat(all_data, ignore_index=True)

    # Save combined DataFrame as CSV
    combined_df.to_csv(local_output_path, index=False)

    # Upload to the new results bucket inside `extracted_queries/`
    blob = results_bucket_ref.blob(processed_gcs_path)
    blob.upload_from_filename(local_output_path)

    print(f"Processed queries saved in GCS: {processed_gcs_path}")

    return processed_gcs_path  # ✅ Output GCS path for next component



# Use BERT to filter similar queries from the Corpus

In [ ]:
@dsl.component(base_image="python:3.9")
def filter_similar_queries(
    results_bucket: str, processed_file_path: str, firestore_project_id: str
) -> list:
    import subprocess
    import sys
    subprocess.run([sys.executable, "-m", "pip", "install",
                   "google-cloud-firestore", "google-cloud-storage",
                   "pandas", "sentence-transformers", "scikit-learn"], check=True)

    import pandas as pd
    from google.cloud import firestore, storage
    import hashlib
    import json
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity

    # Initialize Clients
    storage_client = storage.Client()
    db = firestore.Client(project=firestore_project_id)
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Define Paths
    local_csv_path = f"/tmp/{processed_file_path.split('/')[-1]}"
    results_bucket_ref = storage_client.bucket(results_bucket)
    blob = results_bucket_ref.blob(processed_file_path)

    # Download the processed CSV
    blob.download_to_filename(local_csv_path)
    df = pd.read_csv(local_csv_path)

    # Detect Requirement Column
    req_columns = [col for col in df.columns if 'requirements' in col.lower()]
    if not req_columns:
        raise ValueError("No requirement column found in the processed input.")

    requirement_col = req_columns[0]

    # Prepare data structures for results
    cached_queries = []
    new_queries = []
    query_indices = []
    all_query_data = []

    # Function to find similar queries in Firestore
    def get_similar_cached_response(new_query):
        # Generate the embedding for the new query
        new_query_embedding = model.encode([new_query])[0]

        # Retrieve all cached queries and their embeddings from Firestore
        query_cache_ref = db.collection('query_cache')
        docs = query_cache_ref.stream()

        # Initialize variables to track the most similar query and its response
        most_similar_response = None
        highest_similarity = 0
        source = "None"

        for doc in docs:
            query_data = doc.to_dict()
            cached_embedding = query_data.get('embedding')

            if cached_embedding is None:
                continue

            # Calculate cosine similarity
            similarity = cosine_similarity([new_query_embedding], [cached_embedding])[0][0]

            # If similarity is above threshold, consider it a match
            if similarity > highest_similarity and similarity > 0.85:
                highest_similarity = similarity
                most_similar_response = query_data.get('response')
                source = "Firestore (Similar Match)"

        return most_similar_response, source, new_query_embedding.tolist()

    # Process each query
    for index, row in df.iterrows():
        query = str(row[requirement_col]).strip()
        query_indices.append(index)

        if not query or query.lower() in ["nan", "none", "null"]:
            all_query_data.append({
                "index": index,
                "query": query,
                "response": "No Valid Query Provided",
                "source": "N/A",
                "embedding": None
            })
            continue

        print(f"Processing query {index+1}/{len(df)}: {query[:50]}...")

        # First check for exact match in Firestore
        doc_id = hashlib.sha256(query.encode('utf-8')).hexdigest()
        doc_ref = db.collection('query_cache').document(doc_id)
        doc = doc_ref.get()

        if doc.exists:
            # Add to cached queries list
            response_text = doc.to_dict()['response']
            source = "Firestore (Exact Match)"
            all_query_data.append({
                "index": index,
                "query": query,
                "response": response_text,
                "source": source,
                "embedding": doc.to_dict().get('embedding')
            })
            cached_queries.append(index)
        else:
            # Check for similar queries
            similar_response, source, embedding = get_similar_cached_response(query)

            if similar_response:
                all_query_data.append({
                    "index": index,
                    "query": query,
                    "response": similar_response,
                    "source": source,
                    "embedding": embedding
                })
                cached_queries.append(index)
            else:
                # Add to new queries list that need Discovery Engine processing
                new_queries.append({
                    "index": index,
                    "query": query,
                    "embedding": embedding
                })

    # Save the intermediate results
    intermediate_file = f"/tmp/query_processing_data.json"
    with open(intermediate_file, 'w') as f:
        json.dump({
            "all_query_data": all_query_data,
            "new_queries": new_queries,
            "original_file": processed_file_path
        }, f)

    # Upload to GCS
    intermediate_gcs_path = f"intermediate_data/query_filter_results_{processed_file_path.split('/')[-1].replace('.csv', '.json')}"
    blob = results_bucket_ref.blob(intermediate_gcs_path)
    blob.upload_from_filename(intermediate_file)

    print(f"Found {len(cached_queries)} cached queries and {len(new_queries)} new queries that need processing")

    return [intermediate_gcs_path, processed_file_path]


# Invoke Vertex AI RAG for new or remaining queries

In [ ]:
#@dsl.component(base_image="python:3.9")
@dsl.component(base_image="gcr.io/deeplearning-platform-release/base-cpu")
def process_api_queries(
    results_bucket: str,
    input_data_paths: list,
    firestore_project_id: str,
    bigquery_table: str
) -> str:
    import subprocess
    import sys
    subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir",
                   "google-cloud-firestore", "google-cloud-bigquery",
                   "google-cloud-storage", "pandas", "requests", "google-auth"], check=True)

    import pandas as pd
    from google.cloud import firestore, bigquery, storage
    import requests
    import hashlib
    import time
    import json
    from google.auth import default
    from google.auth.transport.requests import Request


    # Parse input paths
    intermediate_gcs_path = input_data_paths[0]
    processed_file_path = input_data_paths[1]

    # Initialize Clients
    storage_client = storage.Client()
    db = firestore.Client(project=firestore_project_id)
    bigquery_client = bigquery.Client(project=firestore_project_id)

    # Function to get the access token for authentication
    def get_access_token():

      # Assuming this function is set up correctly to get the access token
      credentials, _ = default()
      credentials.refresh(Request())
      return credentials.token


    # Define Discovery Engine API endpoint
    api_url = "https://discoveryengine.googleapis.com/v1alpha/projects/994544669142/locations/global/collections/default_collection/engines/rfp-agent_1722488765661/servingConfigs/default_search:answer"

    # Define Paths
    results_bucket_ref = storage_client.bucket(results_bucket)

    # Download the intermediate data
    local_intermediate_path = f"/tmp/{intermediate_gcs_path.split('/')[-1]}"
    blob = results_bucket_ref.blob(intermediate_gcs_path)
    blob.download_to_filename(local_intermediate_path)

    with open(local_intermediate_path, 'r') as f:
        intermediate_data = json.load(f)

    all_query_data = intermediate_data["all_query_data"]
    new_queries = intermediate_data["new_queries"]

    # Download the original CSV to update at the end
    local_csv_path = f"/tmp/{processed_file_path.split('/')[-1]}"
    blob = results_bucket_ref.blob(processed_file_path)
    blob.download_to_filename(local_csv_path)
    df = pd.read_csv(local_csv_path)

    # Function to call the Discovery Engine API
    def call_discovery_engine(query_text, session_id=None):

      access_token = get_access_token()
      headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
               }

      payload = {
            "query": {
                "text": query_text
            },
            "session": session_id or "",  # Pass empty session_id for the first query
            "answerGenerationSpec": {
                "ignoreAdversarialQuery": True,
                "ignoreNonAnswerSeekingQuery": True,
                "ignoreLowRelevantContent": True,
                "includeCitations": True,
                "promptSpec": {
                    "preamble": (
                        "Please keep the answer concise and summary should be between 100 to max 150 words"
                        "Treat the requirement of input as a question and provide summary"
                        "Dont just provide Yes/No answer. Elaborate on the quesiton and answer thoroougly"
                        "Dont use the names of the files or entities such as banks or a business entity in your answer."
                        "The resulting summary should answer the question or requirement without referring to the documents or sources as a professional sales person would answer the requirement"

                    )
                },
                "modelSpec": {
                    "modelVersion": "preview"
                }
            }
        }

      max_retries = 3
      for attempt in range(max_retries):
        try:
          print(f"Calling Discovery Engine API for: {query_text[:50]}...")
          response = requests.post(api_url, headers=headers, data=json.dumps(payload)) # Recent Update from notebook colab
          response.raise_for_status()
          return response.json()

          # Check if response contains undesired message
          if "summary" in response_json and "A summary could not be generated for your search query" in response_json["summary"]:
            print(f"Skipping query due to invalid response: {query_text[:50]}...")
            return None  # Do not store this response

          return response_json  # Valid response

        except requests.exceptions.RequestException as e:
          print(f"API request attempt {attempt+1} failed: {str(e)}")
          if attempt < max_retries - 1:
            time.sleep(2 * (attempt + 1))  # Exponential backoff
            return None

    # Process new queries in batches
    batch_size = 10  # Adjust based on API quota limits
    session_id = None  # For conversation continuity

    print(f"Processing {len(new_queries)} new queries with Discovery Engine API")

    for i in range(0, len(new_queries), batch_size):
        batch = new_queries[i:i+batch_size]

        for query_item in batch:
            index = query_item["index"]
            query = query_item["query"]
            embedding = query_item["embedding"]

            # Call Discovery Engine
            response_json = call_discovery_engine(query, session_id)

            if response_json:
                response_text = response_json.get("answer", {}).get("answerText", "")
                # Update session ID for conversation continuity if needed
                session_id = response_json.get("session", session_id)
                source = "Discovery Engine"

                # Store in Firestore for future use
                doc_id = hashlib.sha256(query.encode('utf-8')).hexdigest()
                doc_ref = db.collection('query_cache').document(doc_id)
                doc_ref.set({
                    'query': query,
                    'response': response_text,
                    'timestamp': firestore.SERVER_TIMESTAMP,
                    'embedding': embedding
                })

                # Store in BigQuery for analytics
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
                try:
                    bigquery_client.insert_rows_json(
                        bigquery_table,
                        [{"query": query, "response": response_text, "timestamp": timestamp, "source": source}]
                    )
                except Exception as e:
                    print(f"BigQuery insert error: {str(e)}")
            else:
                response_text = "Error: Failed to get response from Discovery Engine API."
                source = "Error"

            # Add to all_query_data
            all_query_data.append({
                "index": index,
                "query": query,
                "response": response_text,
                "source": source,
                "embedding": embedding
            })

        # Add a small delay between batches to avoid rate limiting
        time.sleep(1)

    # Update DataFrame with all responses
    df["AI_Response"] = ""
    df["Response_Source"] = ""

    for query_data in all_query_data:
        idx = query_data["index"]
        df.at[idx, "AI_Response"] = query_data["response"]
        df.at[idx, "Response_Source"] = query_data["source"]

    # Save updated DataFrame
    processed_output_path = f"/tmp/processed_{processed_file_path.split('/')[-1]}"
    df.to_csv(processed_output_path, index=False)

    # Upload to `processed_responses/` inside results bucket
    output_gcs_path = f"processed_responses/{processed_file_path.split('/')[-1]}"
    blob = results_bucket_ref.blob(output_gcs_path)
    blob.upload_from_filename(processed_output_path)

    print(f"Processed {len(all_query_data)} queries total. Results saved to: {output_gcs_path}")

    return output_gcs_path


# Log Component

In [ ]:
@dsl.component(base_image="python:3.9")
def log_pipeline_completion(pipeline_name: str, file_name: str):
    print(f"Pipeline {pipeline_name} completed processing file: {file_name}")
    print(f"Timestamp: {__import__('time').strftime('%Y-%m-%d %H:%M:%S')}")



# Clean up Cache if unwanted data stored in the Firestore

In [ ]:
@dsl.component(base_image="python:3.9")
def delete_invalid_firestore_responses(firestore_project_id: str, collection_name: str) -> int:
    """
    A Kubeflow Pipeline component to delete invalid responses from Firestore.

    Args:
        firestore_project_id (str): Firestore Project ID.
        collection_name (str): Name of the Firestore collection.

    Returns:
        int: Number of deleted records.
    """
    import subprocess
    import sys

    # Install necessary dependencies inside the component
    subprocess.run([sys.executable, "-m", "pip", "install", "--no-cache-dir", "google-cloud-firestore"], check=True)
    from google.cloud import firestore

    # Initialize Firestore Client
    db = firestore.Client(project=firestore_project_id)
    collection_ref = db.collection(collection_name)

    deleted_count = 0  # Track the number of deleted records

    # Fetch all documents in the Firestore collection
    docs = collection_ref.stream()

    for doc in docs:
        doc_data = doc.to_dict()  # Convert document to dictionary

        # Check if 'response' exists and is a string before searching for the phrase
        response_value = doc_data.get("response", "")

        if isinstance(response_value, str) and "A summary could not be generated for your search query" in response_value:
            print(f"Deleting document: {doc.id} due to invalid response")
            collection_ref.document(doc.id).delete()
            deleted_count += 1

    print(f"Total deleted records: {deleted_count}")
    return deleted_count  # Return the number of deleted records


# Exit Handler

In [ ]:
@dsl.component(base_image="python:3.9")
def exit_handler_component(pipeline_name: str, file_name: str, status: str):
    """Logs pipeline completion status and can be used for cleanup tasks."""
    import time

    print(f"Pipeline {pipeline_name} finished with status: {status}")
    print(f"Processed file: {file_name}")
    print(f"Timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")


# Build Pipeline

In [ ]:
@dsl.pipeline(name="optimized_rag_pipeline")
def optimized_rag_pipeline(
    source_bucket: str,
    results_bucket: str,
    file_name: str,
    firestore_project_id: str,
    bigquery_table: str
):
    # ✅ Define an exit handler that logs success/failure
    with dsl.ExitHandler(exit_handler_component(
        pipeline_name="optimized_rag_pipeline",
        file_name=file_name,
        status="FAILED"  # Default to failed; updated below if successful
    )):
        # Step 1: Extract Queries & Save to `extracted_queries/`
        extract_task = extract_queries_from_xlsx(
            source_bucket=source_bucket,
            file_name=file_name,
            results_bucket=results_bucket
        ).set_caching_options(enable_caching=False)

        # Step 2a: Filter queries based on similarity to existing Firestore data
        filter_task = filter_similar_queries(
            results_bucket=results_bucket,
            processed_file_path=extract_task.output,
            firestore_project_id=firestore_project_id
        ).set_caching_options(enable_caching=False)

        # Step 2b: Process remaining queries with Discovery Engine API
        process_task = process_api_queries(
            results_bucket=results_bucket,
            input_data_paths=filter_task.output,
            firestore_project_id=firestore_project_id,
            bigquery_table=bigquery_table
        ).set_caching_options(enable_caching=False)

        # Step 3 :
        delete_invalid_firestore_responses(
        firestore_project_id="veefin-ai-426106",
        collection_name="query_cache").after(process_task)
        """
        # Step 3: Save Final Output
        save_results_to_gcs(
            results_bucket=results_bucket,
            processed_file_path=process_task.output
        ).set_caching_options(enable_caching=False)
        """

    # ✅ If pipeline completes successfully, change status to SUCCESS
    exit_handler_component(
        pipeline_name="optimized_rag_pipeline",
        file_name=file_name,
        status="SUCCESS"
    )


# Pipeline Compilation

In [ ]:
# Compile the new pipeline
from kfp.compiler import Compiler
pipeline_file = "optimized_rag_pipeline.json"
Compiler().compile(optimized_rag_pipeline, pipeline_file)
print(f"Pipeline compiled successfully: {pipeline_file}")


# Pipeline Test

In [ ]:
# Code to run the pipeline
from google.cloud import aiplatform

aiplatform.PipelineJob(
    display_name="optimized-rag-pipeline",
    template_path="optimized_rag_pipeline.json",
    parameter_values={
        "source_bucket": "rfp_data_cloud_function",
        "results_bucket": "rfp_pipeline_results",
        "file_name": "pipeline_test_LOS.xlsx",
        "firestore_project_id": "veefin-ai-426106",
        "bigquery_table": "veefin-ai-426106.rfp_data.rfp_queries_responses_timestamps"
    },
    enable_caching=False  # Disable caching for the entire pipeline
).run()